# Setup Emission Model
Setup the model for calculating the emission. This includes building the line list and configuring which lines are in the wavelength-resolved list.

In [1]:
import urllib
import os
import logging
logging.basicConfig(level=logging.INFO)

import numpy as np
import astropy.units as u
import ChiantiPy.core as ch
import ChiantiPy.tools.util as ch_tools_util

from synthesizAR.atomic import EmissionModel


 using cli
 using CLI for selections
 reading chiantirc file


/opt/anaconda3/envs/synthesizar/lib/python3.5/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
ar_root = '/data/datadrive2/ar_viz/seminar_poc/'

First, get the lines that we want to actually resolve.

In [3]:
new_linelist = [list(filter(None,line.decode('utf-8').strip('\n').split(' '))) \
                for line in urllib.request.urlopen('http://iopscience.iop.org/0004-637X/759/2/141/suppdata/apj446760t2_mrt.txt').readlines()[19:]]
resolved_transitions = []
for line in new_linelist:
    if int(line[0]) == 1 and line[1] == 'EIS':
        el = line[2]
        ion = line[3]
        wvl = float(line[4])
        el_ion = ch_tools_util.spectroscopic2name(el,ion)
        try:
            index = next(index for (index, d) in enumerate(resolved_transitions) if d["name"] == el_ion)
            resolved_transitions[index]['wavelengths'] = (list(resolved_transitions[index]['wavelengths'].value) + [wvl])*u.angstrom
        except StopIteration:
            resolved_transitions.append({'name':el_ion,'wavelengths':[wvl]*u.angstrom})
resolved_transitions.pop(1)

{'name': 'fe_9', 'wavelengths': <Quantity [ 188.497] Angstrom>}

In [4]:
for ion in resolved_transitions:
    print('ion {}'.format(ion['name']))
    ch_ion = ch.ion(ion['name'])
    tmp = []
    for i,w in enumerate(ion['wavelengths'].value):
        new_wvl = min(ch_ion.Wgfa['wvl'],key=lambda x:abs(x-w))
        if abs(new_wvl - w)>0.1:
            raise ValueError('Closest wavelength {} is not within tolerance of {}'.format(new_wvl,w))
        tmp.append(new_wvl)
    tmp = tmp*ion['wavelengths'].unit
    ion['wavelengths'] = tmp

ion si_7
ion fe_10
ion fe_11
ion fe_12
ion fe_13
ion fe_15
ion s_13
ion fe_16
ion ca_17


Now, make a list of *all* of the ions we want to look at.

In [5]:
ion_names = ['si_7','s_13','ca_17'] + ['fe_{}'.format(i) for i in [6,8]+list(range(10,26))]

In [6]:
ion_masterlist = []
wvl_range = [80.0,350.0]
for name in ion_names:
    print('Building list for {}'.format(name))
    tmp = {'name':name}
    ci = ch.ion(name)
    wvls = [w for w in ci.Wgfa['wvl'] if wvl_range[0] <= w <= wvl_range[1]]
    wvls = wvls*u.angstrom
    tmp['wavelengths'] = wvls
    resolved_ion = [rt for rt in resolved_transitions if rt['name']==name]
    if resolved_ion:
        tmp['resolve_wavelength'] = [True if w in resolved_ion[0]['wavelengths'] else False for w in wvls]
    else:
        tmp['resolve_wavelength'] = [False]*len(wvls)
    ion_masterlist.append(tmp)

Building list for si_7
Building list for s_13
Building list for ca_17
Building list for fe_6
Building list for fe_8
Building list for fe_10
Building list for fe_11
Building list for fe_12
Building list for fe_13
Building list for fe_14
Building list for fe_15
Building list for fe_16
Building list for fe_17
Building list for fe_18
Building list for fe_19
Building list for fe_20
Building list for fe_21
Building list for fe_22
Building list for fe_23
Building list for fe_24
Building list for fe_25


Now, create the model and calculate the emissivity.

In [13]:
emiss_model = EmissionModel(ion_masterlist,density=np.logspace(8,11,20)/(u.cm**3),
                            energy_unit='photon',chianti_db_filename=os.path.join(ar_root,'chianti_db.h5'))

INFO:EmissionModel:Using CHIANTI HDF5 database in /data/datadrive2/ar_viz/seminar_poc/chianti_db.h5
INFO:EmissionModel:Building entry for si_7
INFO:EmissionModel:Building elvlc entry for si_7
INFO:EmissionModel:Building wgfa entry for si_7
INFO:EmissionModel:Building scups entry for si_7
INFO:EmissionModel:Building psplups entry for si_7
INFO:EmissionModel:Building entry for s_13
INFO:EmissionModel:Building elvlc entry for s_13
INFO:EmissionModel:Building wgfa entry for s_13
INFO:EmissionModel:Building scups entry for s_13
INFO:EmissionModel:Building psplups entry for s_13
INFO:EmissionModel:Building entry for ca_17
INFO:EmissionModel:Building elvlc entry for ca_17
INFO:EmissionModel:Building wgfa entry for ca_17
INFO:EmissionModel:Building scups entry for ca_17
INFO:EmissionModel:Building psplups entry for ca_17
INFO:EmissionModel:Building entry for fe_6
INFO:EmissionModel:Building elvlc entry for fe_6
INFO:EmissionModel:Building wgfa entry for fe_6
INFO:EmissionModel:Building scups e

In [14]:
emiss_model.calculate_emissivity()

INFO:EmissionModel:Calculating emissivity for ion si_7
INFO:ChIon:Expressing emissivity in units of photons
INFO:ChIon:Calculating level populations.
INFO:ChIon:Calculating emissivity
INFO:EmissionModel:Calculating emissivity for ion s_13
INFO:ChIon:Expressing emissivity in units of photons
INFO:ChIon:Calculating level populations.
INFO:ChIon:Calculating emissivity
INFO:EmissionModel:Calculating emissivity for ion ca_17
INFO:ChIon:Expressing emissivity in units of photons
INFO:ChIon:Calculating level populations.
INFO:ChIon:Calculating emissivity
INFO:EmissionModel:Calculating emissivity for ion fe_6
INFO:ChIon:Expressing emissivity in units of photons
INFO:ChIon:Calculating level populations.
INFO:ChIon:Calculating emissivity
INFO:EmissionModel:Calculating emissivity for ion fe_8
INFO:ChIon:Expressing emissivity in units of photons
INFO:ChIon:Calculating level populations.
INFO:ChIon:Calculating emissivity
INFO:EmissionModel:Calculating emissivity for ion fe_10
INFO:ChIon:Expressing e

Finally, save the model to be reloaded later.

In [16]:
emiss_model.save(savedir=os.path.join(ar_root,'checkpoint_emiss_model'))

INFO:EmissionModel:Saving emission model information in /data/datadrive2/ar_viz/seminar_poc/checkpoint_emiss_model
